In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys, os

In [ ]:
%load_ext qat.core.magic

In [ ]:
%matplotlib inline

In [ ]:
#QPU connection
QLMASS = True
if QLMASS:
    try:
        from qat.qlmaas import QLMaaSConnection
        connection = QLMaaSConnection()
        LinAlg = connection.get_qpu("qat.qpus:LinAlg")
        lineal_qpu = LinAlg()
    except (ImportError, OSError) as e:
        print('Problem: usin PyLinalg')
        from qat.qpus import PyLinalg
        lineal_qpu = PyLinalg()
else:
    print('User Forces: PyLinalg')
    from qat.qpus import PyLinalg
    lineal_qpu = PyLinalg() 

In [ ]:
sys.path.append('/home/cesga/gferro/NEASQC/PhaseAmplification/')

In [ ]:
from QuantumMultiplexors_Module_examples import expectation_loading_data
from PhaseAmplification_Module import load_q_gate

In [ ]:
from AuxiliarFunctions import  get_histogram, postprocess_results, test_bins, run_job
def p(x):
    return x*x
def f(x):
    return np.sin(x)

In [ ]:
#number of Qbits for the circuit
n_qbits = 6
#The number of bins 
m_bins = 2**n_qbits
LowerLimit = 0.0
UpperLimit = 1.0 

X, p_X = get_histogram(p, LowerLimit, UpperLimit, m_bins)
f_X = f(X)
Qprog, P_Gate, R_gate = expectation_loading_data(p_X, f_X)
Q_Gate = load_q_gate(P_Gate, R_gate)

In [ ]:
from iterative_quantum_ae import IterativeQuantumAE

In [ ]:
def getstaff(InputPDF):
    pdf = InputPDF.copy(deep=True)
    #Con esto obtenemos el theta del operador unitario
    pdf['Theta_Unitary'] = 2*np.pi*pdf['Phi']
    #Esto es el ángulo que rota el operador Q en el problema del valor esperador de la función (2*theta)
    #IQAE.results['Theta_Q'] = np.pi*IQAE.results['Phi']
    pdf['Theta'] = np.pi*pdf['Phi']
    #Solo angulos entre [0, pi/2]
    #IQAE.results['Theta'].where(IQAE.results['Theta']< 0.5*np.pi, np.pi-IQAE.results['Theta'], inplace=True)
    pdf['E_p(f)'] = np.sin(pdf['Theta'])**2
    pdf['theta_90'] = pdf['Theta']
    pdf['theta_90'].where(pdf['theta_90']< 0.5*np.pi, np.pi-pdf['theta_90'], inplace=True)    
    return pdf

## 1. Clase Paso a Paso

In [ ]:
iqae_dict = {
    'qpu' : lineal_qpu,
    'cbits_number' : 6,
    #'easy': True,
    'easy': False    
}

In [ ]:
#Instancio la clase
IQAE = IterativeQuantumAE(Qprog, Q_Gate, **iqae_dict)
#Inicializo el programa cuantico
IQAE.init_iqae()

In [ ]:
#puedo pintar el circuito:
circuit = IQAE.q_prog.to_circ(submatrices_only=True)
%qatdisplay circuit

In [ ]:
#Ejecuto un IQAE completo
print('Ejecuto IQAE completo para {} bits clasicos'.format(IQAE.cbits_number))
IQAE.apply_iqae()

In [ ]:
#La clase tiene un método que crea el circuito:
IQAE.get_circuit()
circuit = IQAE.circuit
%qatdisplay circuit

In [ ]:
#A continuacion generamos el job
IQAE.get_job()
#Ejecutamos el job
IQAE.get_job_result()

In [ ]:
#En la propiedad job_result almacenamos la salida del result
IQAE.job_result

In [ ]:
#Para procesar la salida se utiliza el metodo que genera la propiedad results como un DataFrame
IQAE.get_classicalbits()
#
IQAE.results

In [ ]:
#La siguiente funcion nos permite generar las salidas para QAA
pdf = getstaff(IQAE.results)

In [ ]:
pdf

In [ ]:
sum(f_X*p_X)

## 2. Todo de una vez

In [ ]:
iqae_dict = {
    'qpu' : lineal_qpu,
    'cbits_number' : 6,
    'easy': True
}

In [ ]:
#Instancio la clase
IQAE = IterativeQuantumAE(Qprog, Q_Gate, **iqae_dict)
#Ejecuto todo el algoritmo
IQAE.iqae()

In [ ]:
circuit = IQAE.circuit
%qatdisplay circuit

In [ ]:
pdf = getstaff(IQAE.results)

In [ ]:
pdf

In [ ]:
sum(f_X*p_X)

## 3. Misma iteracion n veces

Podemos hacerlo de dos formas: 

1. Iterar n veces sobre la misma clase
2. Proporcionar un nbshots a la entrada con el número de iteraciones que queremos

### 3.1 Iteramos sobre la clase

In [ ]:
%%time
zalo_dict = {
    'qpu' : lineal_qpu,
    'cbits_number' : 6,
    'easy': False    
}
ListaZalo = []
for i in range(5):
    IQAE = IterativeQuantumAE(Qprog, Q_Gate, **zalo_dict)
    IQAE.iqae()
    pdf = getstaff(IQAE.results)
    ListaZalo.append(pdf)
pdf_Zalo = pd.concat(ListaZalo)  
pdf_Zalo.reset_index(drop=True, inplace=True)

In [ ]:
%%time
easy_dict = {
    'qpu' : lineal_qpu,
    'cbits_number' : 6,
    'tipo': True
}
ListaEasy = []
for i in range(5):
    IQAE = IterativeQuantumAE(Qprog, Q_Gate, **easy_dict)
    IQAE.iqae()
    pdf = getstaff(IQAE.results)
    ListaEasy.append(pdf)
pdf_Easy = pd.concat(ListaEasy)   
pdf_Easy.reset_index(drop=True, inplace=True)

In [ ]:
plt.plot(pdf_Zalo['Theta'], pdf_Zalo['Probability'], 'o')
plt.plot(pdf_Easy['Theta'], pdf_Easy['Probability'], 'o')

In [ ]:
plt.plot(pdf_Zalo['E_p(f)'], pdf_Zalo['Probability'], 'o')
plt.plot(pdf_Easy['E_p(f)'], pdf_Easy['Probability'], 'o')

In [ ]:
#Comprobando theta y -theta
eas = pdf_Easy['Theta'].where(pdf_Easy['Theta']<0.5*np.pi, np.pi-pdf_Easy['Theta'])
zal = pdf_Zalo['Theta'].where(pdf_Zalo['Theta']<0.5*np.pi, np.pi-pdf_Zalo['Theta'])
plt.plot(eas, pdf_Easy['Probability'], 'o')
plt.plot(zal, pdf_Zalo['Probability'], 'o')

Voy a grabarlos datos para que quede constancia de que esto funciona como tiene que funcionar:

pdf_Zalo.to_csv('iterative_iqae_zalo_100medidas.csv')#, index=False)
pdf_Easy.to_csv('iterative_iqae_easy_100medidas.csv')#, index=False)

### 3.2 Iteramos utilizando nbshots

In [ ]:
%%time
zalo_dict = {
    'qpu' : lineal_qpu,
    'cbits_number' : 6,
    'easy': False,
    'shots': 100
}
IQAE = IterativeQuantumAE(Qprog, Q_Gate, **zalo_dict)
IQAE.iqae()
pdf_Zalo = getstaff(IQAE.results)

In [ ]:
%%time
easy_dict = {
    'qpu' : lineal_qpu,
    'cbits_number' : 6,
    'easy': True,
    'shots': 100
}
IQAE = IterativeQuantumAE(Qprog, Q_Gate, **easy_dict)
IQAE.iqae()
pdf_Easy = getstaff(IQAE.results)

In [ ]:
pdf_Zalo.sort_values('Theta', inplace=True)
pdf_Easy.sort_values('Theta', inplace=True)

In [ ]:
plt.plot(pdf_Zalo['Theta'], pdf_Zalo['Probability'], 'o-')
plt.plot(pdf_Easy['Theta'], pdf_Easy['Probability'], 'o-', alpha=0.7)
plt.legend(['Normal', 'Easy'])

In [ ]:
plt.plot(pdf_Zalo['theta_90'], pdf_Zalo['Probability'], 'o-')
plt.plot(pdf_Easy['theta_90'], pdf_Easy['Probability'], 'o-', alpha=0.7)
plt.legend(['Normal', 'Easy'])

In [ ]:
plt.plot(pdf_Zalo['E_p(f)'], pdf_Zalo['Probability'], 'o-')
plt.plot(pdf_Easy['E_p(f)'], pdf_Easy['Probability'], 'o-', alpha=0.7)
plt.legend(['Normal', 'Easy'])

## 4. Diferentes Bits Clasicos

In [ ]:
iqae_dict = {
    'qpu' : lineal_qpu,
    #'cbits_number' : 6
}

In [ ]:
Lista = []
Lista_Estadisticas = []
for i in range(1, 11):
    iqae_dict.update({'cbits_number' : i})
    #print(iqae_dict['cbits_number'])
    IQAE = IterativeQuantumAE(Qprog, Q_Gate, **iqae_dict)
    IQAE.iqae()
    getstaff(IQAE.results)
    Lista.append(getstaff(IQAE.results))

In [ ]:
pdf = pd.concat(Lista)

In [ ]:
pdf.reset_index(drop=True, inplace=True)

In [ ]:
plt.plot(pdf['theta_90'], 'o-')

In [ ]:
plt.plot(pdf['E_p(f)'], 'o-')

## Comparo Qiskit

In [ ]:
from qat.lang.AQASM import Program, H, X, PH
nq = 2
nc = 3
q_prog = Program()
q_bits = q_prog.qalloc(nq)
cbits = q_prog.calloc(nc)
for i in range(nq):
    q_prog.apply(X, q_bits[i])
qAux = q_prog.qalloc(1)
q_gate = PH(np.pi/4.0).ctrl()


In [ ]:
circuit = q_prog.to_circ(submatrices_only=True) 

%qatdisplay q_gate 

In [ ]:
from iterative_quantum_ae import IterativeQuantumAE

In [ ]:
iqae_dict = {
    'qpu' : lineal_qpu,
    'cbits_number' : 6,
    'shots': 1000
}

In [ ]:
IQAE = IterativeQuantumAE(q_prog, q_gate, **iqae_dict)
IQAE.iqae()

In [ ]:
cirecuir =IQAE.circuit
%qatdisplay circuit    

In [ ]:
plt.plot(IQAE.results['Phi'], IQAE.results['Probability'])

In [ ]:
IQAE.results['Phi'].describe()

In [ ]:
a = IQAE.results

In [ ]:
a.mean()